In [181]:
import os
import sys
import json
if 'SUMO_HOME' in os.environ:
    sys.path.append(os.path.join(os.environ['SUMO_HOME'], 'tools'))
import sumolib
import importlib
import LLAMAconnect
import subprocess
import shutil
from pprint import pprint

In [142]:
net = sumolib.net.readNet('osm.net.xml')
parkingAreas = list(sumolib.output.parse('osm_stops.add.xml', "parkingArea"))

In [174]:
## Script for generating alternative routes
duaiterate_path = "/usr/share/sumo/tools/assign/duaIterate.py"
def getArgs(net_path, trips_path, additional_path, iterations):
    args = [
        "-n", net_path,
        "-t", trips_path,
        "--additional", additional_path,
        "duarouter--additional-files", additional_path,
        "-l", str(iterations)
    ]
    return args

In [160]:
# Latitude e longitude de locais de interesse
coords = {
    "IC": (-22.813344, -47.063667),
    "FEEC": (-22.820788, -47.066489),
    "HOME": (-22.823781, -47.074111),
    "RU": (-22.817327, -47.071338),
    "IB": (-22.819330, -47.068836),
    "RS": (-22.815302, -47.062603),
    "BC": (-22.816370, -47.071218),
    "FEF": (-22.815240, -47.072680),
    "IFGW": (-22.817940, -47.067007),
    "IMECC": (-22.816385, -47.068329),
    "RA": (-22.822440, -47.065231),
    "IQ": (-22.817794, -47.068351),
    "FEQ": (-22.820118, -47.065556),
    "IG": (-22.813727, -47.069196)
}

In [143]:
radius = 50

In [5]:
def has_parking_spot(lanes, parkingAreas):
    # Example of parkingArea:
    # <parkingArea id="pa-1046248579#0" lane="-1046248579#0_0" roadsideCapacity="94" length="5.00"/>
    # Returns parkingArea id if there is a parking spot in the lane
    lane_ids = [lane.getID() for lane in lanes]
    for park in parkingAreas:
        if park.lane in lane_ids:
            return park.id
    return False

In [151]:
def getClosestEdges(lat, lon, radius):
    # Gets the 10 closest edges to the given lat, lon
    x, y = net.convertLonLat2XY(lon, lat)
    edges = net.getNeighboringEdges(x, y, radius)
    closestEdges = []
    if (len(edges) > 0):
        distanceAndEdges = sorted([(dist, edge) for edge, dist in edges], key=lambda x:x[0])
        closestEdges = [x[1] for x in distanceAndEdges[:10]]
    if len(edges) == 0:
        print(f'No edges found for {lat}, {lon}')
    return closestEdges

In [7]:
def getParkingSpot(lat, lon, radius, parkingAreas):
    # Get the parking spot closest to the given lat, lon
    # Used to set stops for the vehicles

    edges = getClosestEdges(lat, lon, radius)
    # Look for parking spots
    for i in range(len(edges)):
        parking_spot = has_parking_spot(edges[i].getLanes(), parkingAreas)
        if parking_spot:
            return parking_spot

    print("No parking spot found")
    return None

In [8]:
def getPath(location_time_list, parkingAreas):
    # All that is needed to create the trip are the stops (parking areas) and the start and end edges.
    # The duarouter is responsible for finding the path between the edges going through the stops.
    # Here, we get the edges and stops that are going to be sent to LLAMA to create the trip.

    # 'coordinates' is a list of tuples with the latitude and longitude of the points of interest, for example IC, FEEC, IC means that
    # the vehicle will start from IC, stop at a parking lot close to FEEC, and then back to IC.
    # The first and last coordinates should be edges and the others should be parking spots.

    # Departure for 7 is 0, 8 is 100, 9 is 200 and so on
    stop_durations = []
    departures = list(location_time_list.keys())
    stop_durations.append(-1) # Indicates this is an edge and not a parking spot

    path = []
    locations = list(location_time_list.values())
    home = getClosestEdges(*locations[0], radius)[0].getID()
    path.append(home) 
    
    for i in range(1, len(locations)-1):
        stop_durations.append(100 * (departures[i] - departures[i - 1]))
        path.append(getParkingSpot(*locations[i], radius, parkingAreas))
    
    path.append(home)
    stop_durations.append(-1)
    
    return path, stop_durations

In [119]:
def pathToXML(path, vehicleID, veh_type, departure_time, stop_durations):
    # Converts the path to the XML format that LLAMA understands
    xml = f'<trip id="{vehicleID}" type={veh_type} depart="{departure_time}" from="{path[0]}" to="{path[-1]}">\n'
    for i in range(1, len(path)-1):
        xml += f'\t<stop parkingArea="{path[i]}" duration="{stop_durations[i]}"/>\n'
    xml += '</trip>'
    return xml

Getting the routine for the students described

In [91]:
student_info = "A Computer Engineering student, a Computer Science student, two Biology students and one Physics student. One of the biology students loves sports and other biology does not have classes all day long. The Computer Engineering and Computer Science students have lunch and dinner at RU."
importlib.reload(LLAMAconnect)
response = LLAMAconnect.getResponse_trip(student_info)
json_response = json.loads(response)
print(response)

{
   "trip1": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "FEEC", "activity": "classes"},
      "9": {"location": "FEEC", "activity": "classes"},
      "10": {"location": "FEEC", "activity": "classes"},
      "11": {"location": "FEEC", "activity": "classes"},
      "12": {"location": "RU", "activity": "lunch"},
      "13": {"location": "BC", "activity": "study"},
      "14": {"location": "FEEC", "activity": "classes"},
      "15": {"location": "FEEC", "activity": "classes"},
      "16": {"location": "FEEC", "activity": "classes"},
      "17": {"location": "FEEC", "activity": "classes"},
      "18": {"location": "RU", "activity": "dinner"},
      "19": {"location": "HOME", "activity": "relax"},
      "20": {"location": "HOME", "activity": "sleep"}
   },
   "trip2": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "IC", "activity": "classes"},
      "9": {"location": "IC", "activity": "classes"},
      "10": {"locatio

In [188]:
student_info = "Ten students of any course. Not all of them have classes all day long."
importlib.reload(LLAMAconnect)
response = LLAMAconnect.getResponse_trip(student_info)
json_response = json.loads(response)
print(response)

{
   "student1": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "IQ", "activity": "classes"},
      "9": {"location": "IQ", "activity": "classes"},
      "10": {"location": "IQ", "activity": "classes"},
      "11": {"location": "IQ", "activity": "classes"},
      "12": {"location": "RU", "activity": "lunch"},
      "13": {"location": "BC", "activity": "study"},
      "14": {"location": "BC", "activity": "study"},
      "15": {"location": "BC", "activity": "study"},
      "16": {"location": "BC", "activity": "study"},
      "17": {"location": "BC", "activity": "study"},
      "18": {"location": "RA", "activity": "dinner"},
      "19": {"location": "HOME", "activity": "relax"},
      "20": {"location": "HOME", "activity": "sleep"}
   },
   "student2": {
      "7": {"location": "HOME", "activity": "wake up"},
      "8": {"location": "IC", "activity": "classes"},
      "9": {"location": "IC", "activity": "classes"},
      "10": {"location": "IC", "activit

Getting the latitude and longitude for each of the locations

TO DO - If a trip or flow defines at least one stop as child element, the attributes 'from' and 'to' may be omitted. 

In [162]:
location_time_list = []
for j in range(len(json_response)):
    location_time_list.append({})
for i in range(1, len(json_response) + 1):

    trip = json_response[f'student{i}']
    last = coords[trip[f'{7}']['location']]
    location_time_list[i - 1][7] = last
    for j in range(1, len(trip)):
        local = coords[trip[f'{j + 7}']['location']]
        if local != last:
            location_time_list[i - 1][j + 7] = local
            last = local

pprint(location_time_list)

[{7: (-22.823781, -47.074111),
  8: (-22.817794, -47.068351),
  12: (-22.817327, -47.071338),
  13: (-22.81637, -47.071218),
  18: (-22.82244, -47.065231),
  19: (-22.823781, -47.074111)},
 {7: (-22.823781, -47.074111),
  8: (-22.813344, -47.063667),
  12: (-22.817327, -47.071338),
  13: (-22.81637, -47.071218),
  18: (-22.815302, -47.062603),
  19: (-22.823781, -47.074111)},
 {7: (-22.823781, -47.074111),
  8: (-22.816385, -47.068329),
  12: (-22.817327, -47.071338),
  13: (-22.81637, -47.071218),
  18: (-22.82244, -47.065231),
  19: (-22.823781, -47.074111)},
 {7: (-22.823781, -47.074111),
  8: (-22.81794, -47.067007),
  12: (-22.817327, -47.071338),
  13: (-22.81637, -47.071218),
  18: (-22.815302, -47.062603),
  19: (-22.823781, -47.074111)},
 {7: (-22.823781, -47.074111),
  8: (-22.816385, -47.068329),
  12: (-22.817327, -47.071338),
  13: (-22.81637, -47.071218),
  18: (-22.82244, -47.065231),
  19: (-22.823781, -47.074111)},
 {7: (-22.823781, -47.074111),
  8: (-22.820788, -47.0

Taking the locations and turning them into a XML trips, which are written to PATHGEN.trips.xml

In [189]:
def parseTripXML(location_time_list, parkingAreas, departure_times, veh_types, veh_types_per_student):
    # Creates the XML for the trips
    ids = list(veh_types.keys())
    xml = '<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">\n'

    xml += '\n'
    xml += '<!-- Vehicles -->\n'
    for i in range(len(veh_types)):
        id = ids[i]
        veh = veh_types[id]
        xml += f'<vType id="{id}" vClass="{veh["vClass"]}" accel="{veh["accel"]}" decel="{veh["decel"]}" sigma="{veh["sigma"]}" maxSpeed="{veh["maxSpeed"]}">\n'
        xml += '\t<param key="device.rerouting.probability" value="1.0"/>\n'
        xml += '\t<param key="device.rerouting.adaptation-steps" value="18"/>\n'
        xml += '\t<param key="device.rerouting.adaptation-interval" value="10"/>\n'
        xml += '</vType>\n'

    xml += '\n'
    xml += '<!-- Trips -->\n'
    for i in range(len(location_time_list)):
        path, stop_durations = getPath(location_time_list[i], parkingAreas)
        xml += pathToXML(path, f'veh{i + 1}', veh_types_per_student[i], departure_times[i], stop_durations) + '\n'
    xml += '</routes>'
    with open('PATHGEN.trips.xml', 'w') as f:
        f.write(xml)
    return xml

In [190]:
departure_times = []
for i in range(len(location_time_list)):
    departure_times.append(i * 20)

veh_types = {'veh_passenger': {'vClass': 'passenger', 'accel': 2, 'decel': 5, 'sigma': 0.5, 'maxSpeed': 50}} # This is going to be a complete description of every vehicle, still have to choose what
                                                                                                                     # are going to be the parameters
veh_types_per_student = []
for i in range(len(location_time_list)):
    veh_types_per_student.append('veh_passenger')   

xml = parseTripXML(location_time_list, parkingAreas, departure_times, veh_types, veh_types_per_student)
print(xml)

<routes xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:noNamespaceSchemaLocation="http://sumo.dlr.de/xsd/routes_file.xsd">

<!-- Vehicles -->
<vType id="veh_passenger" vClass="passenger" accel="2" decel="5" sigma="0.5" maxSpeed="50">
	<param key="device.rerouting.probability" value="1.0"/>	<param key="device.rerouting.adaptation-steps" value="18"/>	<param key="device.rerouting.adaptation-interval" value="10"/></vType>

<!-- Trips -->
<trip id="veh1" type="veh_passenger" depart="0" from="563505734#2" to="563505734#2">
	<stop parkingArea="pa72508815#1" duration="100"/>
	<stop parkingArea="pa72508815#14" duration="400"/>
	<stop parkingArea="pa72508815#12" duration="100"/>
	<stop parkingArea="pa77674920#0" duration="500"/>
</trip>
<trip id="veh2" type="veh_passenger" depart="20" from="563505734#2" to="563505734#2">
	<stop parkingArea="pa75701374#0" duration="100"/>
	<stop parkingArea="pa72508815#14" duration="400"/>
	<stop parkingArea="pa72508815#12" duration="100"/>
	<stop park

Getting alternative routes for every route found

In [191]:
# Run the duarouter to find alternative routes
# Currently this only works for less than 10 iterations
def getAltRoutes(net_path, trips_path, additional_path, iterations):
    for file in os.listdir('.'):
        if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
            os.remove(file)
    
    subprocess.run(['python3', duaiterate_path] + getArgs(net_path, trips_path, additional_path, iterations))
    for i in range(iterations - 1):
        shutil.rmtree(f'00{i}')
    shutil.move(f'00{iterations-1}/PATHGEN_00{iterations-1}.rou.alt.xml', f'PATHGEN_00{iterations-1}.rou.alt.xml')
    for file in os.listdir('.'):
        if file.startswith('PATHGEN_') and file.endswith('.rou.alt.xml'):
            os.rename(file, f'PATHGEN.rou.alt.xml')

In [192]:
getAltRoutes('osm.net.xml', 'PATHGEN.trips.xml', 'osm_stops.add.xml', 5)

> Executing step 0
>> Running router on PATHGEN.trips.xml
>>> Begin time: 2024-10-10 19:27:02.867391
>>> End time: 2024-10-10 19:27:03.634962
>>> Duration: 0:00:00.767571
<<
>> Running simulation
>>> Begin time: 2024-10-10 19:27:03.635058
>>> End time: 2024-10-10 19:27:05.594027
>>> Duration: 0:00:01.958969
<<
< Step 0 ended (duration: 0:00:02.727154)
------------------

> Executing step 1
>> Running router on 000/PATHGEN_000.rou.alt.xml
>>> Begin time: 2024-10-10 19:27:05.607175
>>> End time: 2024-10-10 19:27:06.403061
>>> Duration: 0:00:00.795886
<<
>> Running simulation
>>> Begin time: 2024-10-10 19:27:06.403187
>>> End time: 2024-10-10 19:27:08.311387
>>> Duration: 0:00:01.908200
<<
< Step 1 ended (duration: 0:00:02.704587)
------------------

> Executing step 2
>> Running router on 001/PATHGEN_001.rou.alt.xml
>>> Begin time: 2024-10-10 19:27:08.338488
>>> End time: 2024-10-10 19:27:09.123969
>>> Duration: 0:00:00.785481
<<
>> Running simulation
>>> Begin time: 2024-10-10 19:27:09.

To use this file, just set PATHGEN.rou.alt.xml as a trips file in osm.sumocfg